In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Courses/RE/project_group_4

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/Courses/RE/project_group_4'
/content


In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing & NLP
import nltk
import re
import string
import gensim
from textblob import Word

from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
data = pd.read_csv('golden_set.csv', encoding='ISO-8859-1')
#data.head()

In [5]:
from ast import literal_eval
# convert 'stringfield' lists to usable structure
features = ['Class']
for feature in features:
    data[feature] = data[feature].apply(literal_eval)

In [6]:
def get_labels(x):
    if isinstance(x, list):
        terms = [i['term'] for i in x]
        if len(terms) > 2:
            terms = terms[:2]
        return terms

In [7]:
data['Class'] = data['Class'].apply(get_labels)

In [8]:
# Data Cleaning
def clean_text(text):
    # remove everything except alphabets
    text = re.sub("[^a-zA-Z]", " ", text)
    # remove whitespaces
    text = ' '.join(text.split())
    text = text.lower()
    
    return text

In [9]:
import re
# creating clean text feature
features = ['User Review']
for feature in features:
    data['clean_' + feature] = data[feature].apply(clean_text)

In [10]:
data['soup'] = data['clean_User Review']
#data.head()

,appVersion,User Review,Date,Rating,Class,clean_User Review,soup
0,2.06,Fundamental flaw Sometimes drops a 4 instead o...,15-01-16,4,[Informative],fundamental flaw sometimes drops a instead of ...,fundamental flaw sometimes drops a instead of ...
1,2.06,Too easy and too slow Being able to undo just ...,18-01-16,2,[Informative],too easy and too slow being able to undo just ...,too easy and too slow being able to undo just ...
2,2.06,"Slow Since latest update, it's moving very slo...",15-02-16,4,[Informative],slow since latest update it s moving very slow...,slow since latest update it s moving very slow...
3,2.06,love it.. good excercise before i go to work..,20-02-16,5,[Non-informative],love it good excercise before i go to work,love it good excercise before i go to work
4,2.06,Keeps crashing Since I upgraded my android thi...,26-02-16,1,[Informative],keeps crashing since i upgraded my android thi...,keeps crashing since i upgraded my android thi...


In [11]:
# stopwords-to compare text data with and without stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)
  
data['soup'] = data['soup'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
stem_process = PorterStemmer()

def tokenize_and_stem(text):
    # tokenization to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    lem = [stem_process.stem(t) for t in filtered_tokens]
    return lem

In [13]:
# Defining a TF-IDF Vectorizer
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), tokenizer=tokenize_and_stem, max_features=10000, use_idf=True)

In [14]:
mb = MultiLabelBinarizer()
mb.fit(data['Class'])

y = mb.transform(data['Class'])

In [15]:
# Basic validation: splitting the data 80-20 train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['soup'], y, test_size=0.2, random_state=55)

In [16]:
# Tf-Idf transformation 
xtrain_tfidf = tfidf_vec.fit_transform(X_train)
xtest_tfidf = tfidf_vec.transform(X_test)
xtrain_tfidf.shape


(1266, 10000)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

#EMNB Classifier

In [18]:

multinomial_clf = MultinomialNB(alpha=0.1)
oneVsRest_rf = OneVsRestClassifier(multinomial_clf)

oneVsRest_rf.fit(xtrain_tfidf, y_train)

OneVsRestClassifier(estimator=MultinomialNB(alpha=0.1))

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

y_pred1 = oneVsRest_rf.predict(xtest_tfidf)

def get_performance(y_test, y_pred1):
    # Evaluate Performance
    accuracy = round(accuracy_score(y_test, y_pred1) * 100, 2)
    # Get precision, recall, f1 scores
    precision, recall, f1score, support = score(y_test, y_pred1, average='micro')
    return accuracy, precision, recall, f1score
# Performance metrics
accuracy, precision, recall, f1score = get_performance(y_test, y_pred1)
print(f'Test Accuracy Score of SemisupervisedNB.: % {accuracy}')
print(f'Precision : {precision}')
print(f'Recall    : {recall}')
print(f'F1-score   : {f1score}')

Test Accuracy Score of SemisupervisedNB.: % 79.18
Precision : 0.7917981072555205
Recall    : 0.7917981072555205
F1-score   : 0.7917981072555206


In [20]:
def infer_tags(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text_vec = tfidf_vec.transform([text])
    y_pred1 = oneVsRest_rf.predict(text_vec)
    return mb.inverse_transform(y_pred1)

In [23]:
new_data = pd.read_csv('Hangar.csv', encoding='ISO-8859-1')
#new_data.head()

In [24]:
sentences = []
Prediction_Label = []

for i,_ in enumerate(new_data['User Review']):
    sentenc_store = new_data['User Review'][i]
    sentences.append(sentenc_store)
    Prediction_store = infer_tags(new_data['User Review'][i])
    Prediction_Label.append(Prediction_store)
    print("sentence: ", sentenc_store)
    print("Predicted Label: ",  Prediction_store, "\n")

sentence:  Wasnt quite what I thought it was when I downloaded it but it is pretty cool. I was hoping it would replace my other  notification app but it probably wont. Ill keep it around and see how it learns my favs. Thanks!
Predicted Label:  [('Informative',)] 

sentence:  After a couple days of using hangar, I cant imagine not having its intelligent shortcuts in my notification drawer. Great feature! My only problem with the app is being forced to see the horrifying number of hours I spend on my device playing plants vs zombies. Cant wait to see what this dev comes up with next!
Predicted Label:  [('Informative',)] 

sentence:  A whitelist would be just as useful as the blacklist.
Predicted Label:  [('Informative',)] 

sentence:  Great app by a great dev
Predicted Label:  [('Non-informative',)] 

sentence:  All that space in the notification area was going to waste. Now I can clean up my home screen of icons. Thank you!
Predicted Label:  [('Informative',)] 

sentence:  Great stylish

In [25]:
dict = {'sentence': sentences, 'Prediction_label': Prediction_Label}
df = pd.DataFrame(dict)
df

,sentence,Prediction_label
0,Wasnt quite what I thought it was when I downl...,"[(Informative,)]"
1,"After a couple days of using hangar, I cant im...","[(Informative,)]"
2,A whitelist would be just as useful as the bla...,"[(Informative,)]"
3,Great app by a great dev,"[(Non-informative,)]"
4,All that space in the notification area was go...,"[(Informative,)]"
...,...,...
426,Excelente!,"[(Informative,)]"
427,TrÃ¨s bonne application ! Peut-Ãªtre faudrait ...,"[(Informative,)]"
428,After Phone unlock shows wrong number of Apps ...,"[(Informative,)]"
429,Badass af,"[(Informative,)]"


In [26]:
df.to_csv('predicted/Hangar_predicted.csv')

FileNotFoundError: ignored